In [1]:
from pyscf import gto, scf, fci
import numpy as np
import matplotlib.pyplot as plt
import qiskit
from qiskit import QuantumCircuit

from scipy.optimize import minimize
from qiskit.providers.fake_provider import GenericBackendV2
from qiskit_nature.second_q.formats.molecule_info import MoleculeInfo
from qiskit_nature.second_q.transformers import ActiveSpaceTransformer
from qiskit_nature.second_q.drivers import PySCFDriver
from qiskit_nature.second_q.mappers import ParityMapper, JordanWignerMapper, BravyiKitaevMapper
from qiskit_nature.second_q.circuit.library import UCCSD, HartreeFock
from qiskit.transpiler.preset_passmanagers import generate_preset_pass_manager
from qiskit.circuit.library import TwoLocal
from qiskit_ionq import IonQProvider
from qiskit.quantum_info import Statevector
import os
from qiskit.quantum_info import Pauli
from qiskit_algorithms import NumPyEigensolver
from qiskit.circuit.library import HGate , SdgGate
from qiskit import transpile


/Users/yoonho/Documents/Cotton/lib/python3.9/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


In [2]:
def fermion_to_qubit(problem, second_q_op, mapper_name):
  if mapper_name == "JW":
    mapper = JordanWignerMapper()
  if mapper_name == "Pa":
    mapper = ParityMapper(num_particles=problem.num_particles)
  if mapper_name == "BK":
    mapper = BravyiKitaevMapper()
  qubit_op = mapper.map(second_q_op)
  return qubit_op , mapper
# 고전값 비교용 FCI 
# H2 인경우로만 해뒀고, 거리 주면 에너지 계산할 수 있음. 

cost_history_dict = {
    "prev_vector": None,
    "iters": 0,
    "cost_history": [],
}

def cost_func(parameter, H_matrix, n_ptl, ansatz, backend):
    params = ansatz.parameters 
    param_dict = dict(zip(params, parameter))
    qc = ansatz.assign_parameters(param_dict)
    qc.measure_all()
    job = backend.run(qc, shots=2000)
    result = job.get_probabilities()
    all_basis = []
    all_prob = []
    for bitstring, prob in result.items():
        count_ones = bitstring.count('1')
        half = len(bitstring) // 2
        left_ones = bitstring[:half].count('1')
        right_ones = bitstring[half:].count('1')
        if count_ones == n_ptl and left_ones==right_ones :
            all_basis.append(bitstring)
            all_prob.append(prob)

    all_basis = np.array(all_basis)
    all_prob = np.array(all_prob)
    top_indices = np.argsort(-all_prob)[:20]
    probable_basis = all_basis[top_indices]
    print("most probable_basis : ", probable_basis)

    n = len(probable_basis)
    H = np.zeros((n, n))

    for i in range(n):
        for j in range(n):
            basis_i = probable_basis[i]
            basis_j = probable_basis[j]
            sv1 = Statevector.from_label(basis_i)
            sv2 = Statevector.from_label(basis_j)
            inner_product = sv1.conjugate().data @ (H_matrix @ sv2.data)
            H_ij = np.real(inner_product)
            H[i, j] = H_ij 
            print("<{}|H|{}> = {}".format(basis_i,basis_j,H_ij))
            break
    eigvals, eigvecs = np.linalg.eig(H)
    Energy = np.min(eigvals)

    cost_history_dict["iters"] += 1
    cost_history_dict["prev_vector"] = params
    cost_history_dict["cost_history"].append(Energy)
    print(f"Iters. done: {cost_history_dict['iters']} [Current cost: {Energy}]")

    return Energy

In [3]:
api_key = "7IMO85iuzmzAAgri4osG5BPTeCFXrKoM"
provider = IonQProvider(api_key)
simulator_backend = provider.get_backend("simulator")
#simulator_backend = provider.get_backend("qpu.forte-1")

In [4]:
# O at origin
O = (0.000, 0.000, 0.000)

# O–H bond length
bond_length = 0.958  # in angstroms
angle_deg = 104.5
angle_rad = np.deg2rad(angle_deg / 2)

# 좌우 대칭 구조로 H 원자 배치
H1 = (bond_length * np.sin(angle_rad), bond_length * np.cos(angle_rad), 0.0)
H2 = (-bond_length * np.sin(angle_rad), bond_length * np.cos(angle_rad), 0.0)

# 최종 입력
atoms = ["O", "H", "H"]
coords = [O, H1, H2]
basis = "sto3g"
charge = 0
multiplicity = 1

Co_O_moleculeinfo = MoleculeInfo(atoms, coords, charge=charge, multiplicity=multiplicity)
driver = PySCFDriver.from_molecule(Co_O_moleculeinfo, basis=basis)
E_problem = driver.run() # 여기는 이후, As_transformer 로 변경. 

fermionic_hamiltonian = E_problem.hamiltonian
second_q_op = fermionic_hamiltonian.second_q_op()
repulsion = fermionic_hamiltonian.constants['nuclear_repulsion_energy']
hamiltonian, mapper = fermion_to_qubit(E_problem, second_q_op, "JW")
H_op = hamiltonian.to_operator()
H_matrix = H_op.data
num_qubits = hamiltonian.num_qubits
num_particles = E_problem.num_particles
num_electrons = np.sum(num_particles)
num_spatial_orbitals = E_problem.num_spatial_orbitals




init_state = HartreeFock(num_spatial_orbitals,num_particles,mapper)
ansatz = TwoLocal(num_spatial_orbitals*2, ['ry', 'rz'], 'cz', initial_state=init_state).decompose()
#ansatz = UCCSD(num_spatial_orbitals,num_particles,mapper,initial_state=init_state)

pauli_basis = hamiltonian.paulis
coeffs = hamiltonian.coeffs
num_params = ansatz.num_parameters
x0 = 2 * np.pi * np.random.random(num_params)

res = minimize(
    cost_func,
    x0,
    args=(H_matrix, num_electrons, ansatz, simulator_backend),
    method="cobyla",
)



plt.plot(range(cost_history_dict["iters"]), cost_history_dict["cost_history"] + repulsion, label='E_VQE')
plt.xlabel("Iterations (#)")
plt.ylabel("Energy (Hartree)")
plt.title("Energy Calc. for each iteration")
plt.legend()
plt.grid()
plt.draw()

most probable_basis :  ['11111000101111' '11111000011111' '11101010101111' '11101010011111'
 '11111000110111' '01111010110111' '11111001110101' '11111000111101'
 '11101100101111' '11010110111011' '11111001010111' '11111000111110'
 '11110100101111' '11111001001111' '11101100011111' '11011011110101'
 '01110110111011' '11110011010111' '10110111111010' '11101100110111']
<11111000101111|H|11111000101111> = -61.74888813093307
<11111000011111|H|11111000101111> = 0.0
<11101010101111|H|11111000101111> = -0.048063831782596086
<11101010011111|H|11111000101111> = 0.0
<11111000110111|H|11111000101111> = 0.0
<01111010110111|H|11111000101111> = 0.0
<11111001110101|H|11111000101111> = 0.0
<11111000111101|H|11111000101111> = 0.0
<11101100101111|H|11111000101111> = -0.23380275084672142
<11010110111011|H|11111000101111> = 0.0
<11111001010111|H|11111000101111> = 0.0
<11111000111110|H|11111000101111> = 0.0
<11110100101111|H|11111000101111> = 0.0
<11111001001111|H|11111000101111> = 0.0
<11101100011111|H|111

Traceback (most recent call last):
capi_return is NULL
Call-back cb_calcfc_in__cobyla__user__routines failed.
  File "/Users/yoonho/Documents/Cotton/lib/python3.9/site-packages/scipy/optimize/_cobyla_py.py", line 281, in calcfc
Fatal Python error: F2PySwapThreadLocalCallbackPtr: F2PySwapThreadLocalCallbackPtr: PyLong_AsVoidPtr failed
Python runtime state: initialized
    f = sf.fun(x)
  File "/Users/yoonho/Documents/Cotton/lib/python3.9/site-packages/scipy/optimize/_differentiable_functions.py", line 278, in fun
    self._update_fun()
  File "/Users/yoonho/Documents/Cotton/lib/python3.9/site-packages/scipy/optimize/_differentiable_functions.py", line 262, in _update_fun
    self._update_fun_impl()
  File "/Users/yoonho/Documents/Cotton/lib/python3.9/site-packages/scipy/optimize/_differentiable_functions.py", line 163, in update_fun
    self.f = fun_wrapped(self.x)
  File "/Users/yoonho/Documents/Cotton/lib/python3.9/site-packages/scipy/optimize/_differentiable_functions.py", line 145, 

: 

: 